In [1]:
import tensorflow as tf
if __name__=="__main__":
    tf.enable_eager_execution()

In [2]:
import dataloader as dl
import model_alexnet as model

**Define Configuration**

In [3]:
#Config
#CIFAR-10
DATASET_PATH = "../../Datasets/cifar-10-batches-py/"
LR = 1e-5
EPOCHS = 40
CLASSES = 10
alexnetmodel = model.Alexnet(CLASSES)

In [4]:
dataset = dl.cifar10_loader(DATASET_PATH,buffer_size=1024,batch_size=256)

**Define Loss and Compile Model**

In [5]:
class softmax_cross_entropy:
    def __call__(self,onehot_labels,logits):
        return tf.losses.softmax_cross_entropy(onehot_labels,logits)

In [31]:
opt = tf.train.AdamOptimizer(learning_rate=LR)
criterian = softmax_cross_entropy()
alexnetmodel.compile(optimizer=opt,loss=criterian,metrics=["accuracy"])

### Compute Accuracy

In [46]:
def calc_Accuracy(model,dataset):
    accuracy = 0.0
    total_points = 0.0
    for i,(x,y) in enumerate(dataset('test')):
        y = np.argmax(y.numpy(),axis=1).reshape(-1,1)
        yhat = np.argmax(model.predict(x),axis=1).reshape(-1,1)
        total_points+=y.shape[0]
        col_idx = np.arange(y.shape[0])
        accuracy+=np.sum(y==yhat)
        print('Evaluating %d'%(i),end='\r')
    return accuracy/total_points

### Train the Model

In [44]:
import time

In [43]:
p_acc = 0
for e in range(EPOCHS):
    epoch_loss = 0
    for i,(x,y) in enumerate(dataset('train')):
        epoch_loss +=alexnetmodel.fit(x=x,y=y,epochs=1,verbose=0,batch_size=128).history['loss'][0]
        print('Epoch %d batch %d'%(e+1,i+1),end='\r')
        
    #Reduce the learning rate after 25 epochs
    if (e+1)%25 ==0:
        opt.__dict__['_lr']/=2
    
    #Compute Test/Validation Accuracy after every 2 epochs
    if (e+1)%2 == 0:
        accuracy = calc_accuracy(alexnetmodel,dataset)
        if accuracy > p_acc:
            alexnetmodel.save_weights(filepath="alexnet"+'.h5')
            p_acc = accuracy
        print("Time %s Epoch %d, loss %f acc %f"%(time.ctime(),e+1,epoch_loss,accuracy))
        continue
        
print('Time %s Epoch %d loss %f'%(time.ctime(),e+1,epoch_loss))

KeyboardInterrupt: 